### Make stylizers, metric functions, and load dataset

In [1]:
import SAM
import os
from styleTransfer import StyleTransferLinear, StyleTransferPartialConv
import utils
import numpy as np
import pandas as pd

stylizer_ref = StyleTransferLinear(device="cuda")
stylizer = StyleTransferPartialConv(device="cuda")

sam_data_dir = "../SA-1B-Dataset/"
style_dir = "styles/"

sam_files = os.listdir(sam_data_dir)
sam_jsons = [f for f in sam_files if f.endswith(".json")]
#sam_jsons.sort()
sam_images = [f for f in sam_files if f.endswith(".jpg") or f.endswith(".png")]
#sam_images.sort()

style_ims = os.listdir(style_dir)
                           


In [2]:
import metrics

def computeAllMetrics(content, style, mask, result_stm, result_pc):

    # Between content and masked region
    content_np = utils.tensorToImage(content)
    mask_np = utils.tensorToMask(mask)
    gray_range = metrics.computeColorRange(content_np, in_gray=True)
    gray_range_masked = metrics.computeColorRange(content_np, mask_np, in_gray=True)
    gray_average = metrics.computeColorAverage(content_np, in_gray=True)
    gray_average_masked = metrics.computeColorAverage(content_np, mask_np, in_gray=True)
    gray_std = metrics.computeColorSTD(content_np, in_gray=True)
    gray_std_masked = metrics.computeColorSTD(content_np, mask_np, in_gray=True)
    gray_skew = metrics.computeColorSkewness(content_np, in_gray=True)
    gray_skew_masked = metrics.computeColorSkewness(content_np, mask_np, in_gray=True)
    gray_kurt = metrics.computeColorKurtosis(content_np, in_gray=True)
    gray_kurt_masked = metrics.computeColorKurtosis(content_np, mask_np, in_gray=True)
    gray_emd = metrics.computeEarthMoversDistance(content_np, content_np, None, mask_np, in_gray=True)
    color_sliced_emd = metrics.computeSlicedWassersteinDistance(content_np, content_np, None, mask_np)

    style_np = utils.tensorToImage(style)
    result_stm_np = utils.tensorToImage(result_stm)
    result_pc_np = utils.tensorToImage(result_pc)

    # Between style, style-then-mask, and partial conv
    gray_range_style = metrics.computeColorRange(style_np, in_gray=True)
    gray_range_result_stm = metrics.computeColorRange(result_stm_np, mask_np, in_gray=True)
    gray_range_result_pc = metrics.computeColorRange(result_pc_np, mask_np, in_gray=True)

    gray_average_style = metrics.computeColorAverage(style_np, in_gray=True)
    gray_average_result_stm = metrics.computeColorAverage(result_stm_np, mask_np, in_gray=True)
    gray_average_result_pc = metrics.computeColorAverage(result_pc_np, mask_np, in_gray=True)

    gray_std_style = metrics.computeColorSTD(style_np, in_gray=True)
    gray_std_result_stm = metrics.computeColorSTD(result_stm_np, mask_np, in_gray=True)
    gray_std_result_pc = metrics.computeColorSTD(result_pc_np, mask_np, in_gray=True)

    gray_emd_result_stm = metrics.computeEarthMoversDistance(style_np, result_stm_np, None, mask_np, in_gray=True)
    gray_emd_result_pc = metrics.computeEarthMoversDistance(style_np, result_pc_np, None, mask_np, in_gray=True)

    color_sliced_emd_result_stm = metrics.computeSlicedWassersteinDistance(style_np, result_stm_np, None, mask_np)
    color_sliced_emd_result_pc = metrics.computeSlicedWassersteinDistance(style_np, result_pc_np, None, mask_np)

    result_metrics = {
        "gray_range": gray_range,
        "gray_range_masked": gray_range_masked,
        "gray_average": gray_average,
        "gray_average_masked": gray_average_masked,
        "gray_std": gray_std,
        "gray_std_masked": gray_std_masked,
        "gray_skew": gray_skew,
        "gray_skew_masked": gray_skew_masked,
        "gray_kurt": gray_kurt,
        "gray_kurt_masked": gray_kurt_masked,
        "gray_emd": gray_emd,
        "color_sliced_emd": color_sliced_emd,

        "gray_range_style": gray_range_style,
        "gray_range_result_stm": gray_range_result_stm,
        "gray_range_result_pc": gray_range_result_pc,

        "gray_average_style": gray_average_style,
        "gray_average_result_stm": gray_average_result_stm,
        "gray_average_result_pc": gray_average_result_pc,

        "gray_std_style": gray_std_style,
        "gray_std_result_stm": gray_std_result_stm,
        "gray_std_result_pc": gray_std_result_pc,

        "gray_emd_result_stm": gray_emd_result_stm,
        "gray_emd_result_pc": gray_emd_result_pc,

        "color_sliced_emd_result_stm": color_sliced_emd_result_stm,
        "color_sliced_emd_result_pc": color_sliced_emd_result_pc
    }

    return result_metrics


In [3]:
df = pd.DataFrame(columns=["content", "style", "mask_num", "mask_area", "mask_ratio",
                            "gray_range", "gray_range_masked",
                            "gray_average", "gray_average_masked",
                            "gray_std", "gray_std_masked",
                            "gray_skew", "gray_skew_masked",
                            "gray_kurt", "gray_kurt_masked",
                            "gray_emd", "color_sliced_emd",
    
                            "gray_range_style", "gray_range_result_stm", "gray_range_result_pc",
                            "gray_average_style", "gray_average_result_stm", "gray_average_result_pc",
                            "gray_std_style", "gray_std_result_stm", "gray_std_result_pc",
                            "gray_emd_result_stm", "gray_emd_result_pc",
                            "color_sliced_emd_result_stm", "color_sliced_emd_result_pc",
                            
                            "style_loss_stm", "style_loss_pc",
                            ])

vgg_model = metrics.getVGGmodel(device="cuda")

### Stylize each image and compute metrics

In [ ]:
from tqdm import trange

num_iters = 500
for i in trange(1,num_iters+1):
    #sam_json_fn = sam_jsons[i]
    sam_json_fn = "sa_" + str(i) + ".json"
    if not sam_json_fn in sam_jsons:
        print("Skipping", sam_json_fn, "as it does not exist in the dataset.")
        continue

    sam_json_path = os.path.join(sam_data_dir, sam_json_fn)
    sam_image_path = os.path.join(sam_data_dir, sam_json_fn.replace(".json", ".jpg"))
    
    if not os.path.exists(sam_image_path):
        continue
    
    # choose a random mask and style
    sam_json = SAM.loadMaskJSON(sam_json_path)
    #print(len(sam_json['annotations']), "masks found in", sam_json_fn)
    num_masks = min(len(sam_json['annotations']),100)  # Limit to 100 masks for performance

    # Print mask areas in order
    #for mask_num in range(num_masks):
    #    mask_area = sam_json['annotations'][mask_num]['area']
    #    print(f"Mask {mask_num}: Area = {mask_area}")

    # Attempt to find a mask with a sufficient area
    image_width = sam_json['image']['width']
    image_height = sam_json['image']['height']
    total_pixels = image_width * image_height

    # Random order of numbers from 0 to num_masks-1
    mask_indices = np.arange(num_masks,dtype=np.int32)
    np.random.shuffle(mask_indices)

    for index in mask_indices:
        mask_num = int(index)
        mask_area = sam_json['annotations'][mask_num]['area']
        mask_ratio = mask_area / total_pixels
        if mask_ratio < 0.02: # Only consider masks that cover more than 2% of the image
            continue
        else:
            break

    mask = SAM.loadIndices(sam_json, mask_num)[0]

    content_im = utils.loadImage(sam_image_path)

    style_im_choice = np.random.choice(style_ims)
    style_im_path = os.path.join(style_dir, style_im_choice)
    style_im = utils.loadImage(style_im_path)
        
    result_stm = stylizer_ref(content_im, style_im, mask)
    result_pc = stylizer(content_im, style_im, mask)

    result_metrics = computeAllMetrics(content_im, style_im, mask, result_stm, result_pc)

    style_loss_stm = metrics.computePerceptualStyleLoss(result_stm, style_im, mask, vgg_model)
    style_loss_pc = metrics.computePerceptualStyleLoss(result_pc, style_im, mask, vgg_model)

    df = pd.concat([df, pd.DataFrame({
        "content": sam_json_fn,
        "style": style_im_choice,
        "mask_num": mask_num,
        "mask_area": mask_area,
        "mask_ratio": mask_ratio,
        "gray_range": result_metrics["gray_range"],
        "gray_range_masked": result_metrics["gray_range_masked"],
        "gray_average": result_metrics["gray_average"],
        "gray_average_masked": result_metrics["gray_average_masked"],
        "gray_std": result_metrics["gray_std"],
        "gray_std_masked": result_metrics["gray_std_masked"],
        "gray_skew": result_metrics["gray_skew"],
        "gray_skew_masked": result_metrics["gray_skew_masked"],
        "gray_kurt": result_metrics["gray_kurt"],
        "gray_kurt_masked": result_metrics["gray_kurt_masked"],
        "gray_emd": result_metrics["gray_emd"],
        "color_sliced_emd": result_metrics["color_sliced_emd"],
        "gray_range_style": result_metrics["gray_range_style"],
        "gray_range_result_stm": result_metrics["gray_range_result_stm"],
        "gray_range_result_pc": result_metrics["gray_range_result_pc"],
        "gray_average_style": result_metrics["gray_average_style"],
        "gray_average_result_stm": result_metrics["gray_average_result_stm"],
        "gray_average_result_pc": result_metrics["gray_average_result_pc"],
        "gray_std_style": result_metrics["gray_std_style"],
        "gray_std_result_stm": result_metrics["gray_std_result_stm"],
        "gray_std_result_pc": result_metrics["gray_std_result_pc"],
        "gray_emd_result_stm": result_metrics["gray_emd_result_stm"],
        "gray_emd_result_pc": result_metrics["gray_emd_result_pc"],
        "color_sliced_emd_result_stm": result_metrics["color_sliced_emd_result_stm"],
        "color_sliced_emd_result_pc": result_metrics["color_sliced_emd_result_pc"],
        "style_loss_stm": style_loss_stm,
        "style_loss_pc": style_loss_pc
        
    })], ignore_index=True)

print(df)

100%|██████████| 75/75 [1:08:38<00:00, 54.92s/it]

         content         style mask_num mask_area  mask_ratio  gray_range  \
0      sa_1.json   style-2.jpg       52     94493    0.030580    0.999900   
1      sa_2.json   style-1.jpg       63    359568    0.106539    0.911315   
2      sa_3.json  style-10.jpg       85   1313042    0.389222    0.999900   
3      sa_4.json   style-1.jpg       81    171529    0.050823    0.933476   
4      sa_5.json   style-6.jpg        8   2297012    0.680596    0.976838   
..           ...           ...      ...       ...         ...         ...   
495  sa_496.json   style-8.jpg       80    393465    0.116582    0.999900   
496  sa_497.json   style-6.jpg       81    363704    0.107764    0.999900   
497  sa_498.json   style-0.jpg       89      7355    0.002179    0.999900   
498  sa_499.json   style-9.jpg       49    201230    0.059624    0.999900   
499  sa_500.json   style-4.jpg       51    136218    0.040361    0.999006   

     gray_range_masked  gray_average  gray_average_masked  gray_std  ...  \

In [7]:
# Save the DataFrame to a CSV file
df.to_csv("outputs/style_transfer_metrics.csv", index=False)

### Generate summary statistics

In [8]:
print("Average Grayscale EMD (STM):", df["gray_emd_result_stm"].mean())
print("Average Grayscale EMD (PC):", df["gray_emd_result_pc"].mean())
print("Average Color Sliced EMD (STM):", df["color_sliced_emd_result_stm"].mean())
print("Average Color Sliced EMD (PC):", df["color_sliced_emd_result_pc"].mean())
print("Average Style Loss (STM):", df["style_loss_stm"].mean())
print("Average Style Loss (PC):", df["style_loss_pc"].mean())

Average Grayscale EMD (STM): 0.12213961797303154
Average Grayscale EMD (PC): 0.0855057935550118
Average Color Sliced EMD (STM): 0.16830194202697885
Average Color Sliced EMD (PC): 0.11813415935531874
Average Style Loss (STM): 760.131100692749
Average Style Loss (PC): 449.24159561920163
